In [2]:
import time
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np
from contextlib import contextmanager
import gc 
from util import s_to_time_format, string_to_datetime, hour_to_range, kfold_lightgbm, kfold_xgb
from util import rolling_stats_target_by_cols
#from util import _time_elapsed_between_last_transactions,time_elapsed_between_last_transactions
#from util import num_transaction_in_past_n_days
#from util import add_auto_encoder_feature
#from util import group_target_by_cols_split_by_users
from time import strftime, localtime
import logging
import sys
from config import Configs

In [3]:
def group_target_by_cols(df_train, df_test, recipe):
    df = pd.concat([df_train, df_test], axis = 0)
    for m in range(len(recipe)):
        cols = recipe[m][0]
        for n in range(len(recipe[m][1])):
            target = recipe[m][1][n][0]
            method = recipe[m][1][n][1]
            name_grouped_target = method+"_"+target+'_BY_'+'_'.join(cols)
            tmp = df[cols + [target]].groupby(cols).agg(method)
            tmp = tmp.reset_index().rename(index=str, columns={target: name_grouped_target})
            df_train = df_train.merge(tmp, how='left', on=cols)
            
            df_test = df_test.merge(tmp, how='left', on=cols)

        # reduced memory    
        del tmp
        gc.collect()
    
    return df_train, df_test

In [6]:
# 只看5000筆
df_train = pd.read_csv("train.csv").head(5000)


In [7]:
df_train

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey
0,6881,113261,38038,513.80,5,0,N,0,N,N,...,0,33,172652.0,457,59333,N,0,102,0,516056
1,0,134508,45725,465.62,5,0,N,2,N,N,...,0,9,105114.0,451,0,N,5817,102,0,4376
2,6881,15408,188328,513.80,5,0,N,0,N,N,...,0,6,152458.0,457,59333,N,0,102,0,483434
3,6716,157159,29967,1016.11,5,62,N,5,N,N,...,0,5,172946.0,247,50436,N,3281,102,0,1407164
4,5975,105985,81305,713.66,5,62,N,4,N,N,...,0,6,182129.0,263,93775,N,5817,102,0,1051004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,6881,29914,210026,513.80,5,0,N,0,N,N,...,0,56,64519.0,457,59369,N,0,102,0,541518
4996,0,21984,205835,465.62,5,0,N,2,N,N,...,0,46,105823.0,451,0,N,5817,102,0,22138
4997,6767,68907,87723,841.46,5,62,N,5,N,N,...,0,56,122934.0,241,19197,N,5817,102,0,1224434
4998,6769,156310,201805,346.96,5,62,N,5,N,N,...,0,59,185421.0,251,81372,N,5858,102,0,462689


In [11]:
df_test = pd.read_csv("test.csv").head(5000)

In [12]:
df_test

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey
0,6881,163188,116799,513.80,5,0,N,0,N,N,...,0,102,215328.0,457,59360,N,0,102,0,592489
1,6881,163188,116799,513.80,5,0,N,0,N,N,...,0,102,222007.0,457,59360,N,0,102,0,592452
2,6881,163188,116799,513.80,5,0,N,0,N,N,...,0,100,170013.0,457,59360,N,0,102,0,590212
3,6881,163188,116799,513.80,5,0,N,0,N,N,...,0,100,165914.0,457,59360,N,0,102,0,590209
4,6881,163188,116799,513.80,5,0,N,0,N,N,...,0,102,215311.0,457,59360,N,0,102,0,592488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3493,40382,85290,490.14,5,71,Y,8,N,N,...,0,98,170724.0,360,51357,N,413,94,0,796397
4996,6767,40382,85290,74.77,5,62,Y,8,N,N,...,0,114,161132.0,441,20181,N,5817,102,0,1258007
4997,6767,40382,85290,39.24,5,62,Y,8,N,N,...,0,100,162321.0,441,20181,N,5817,102,0,1250889
4998,6769,40382,85290,168.20,5,62,Y,8,N,N,...,0,100,162436.0,373,81206,N,5817,102,0,327739


In [13]:
# axis = 0 表示index 所以是往下concat
df = pd.concat([df_train, df_test], axis = 0)
df

/Users/Spencer/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey
0,6881,113261,38038,513.80,5,0,N,0,N,N,...,0,33,172652.0,457,59333,N,0,102,0,516056
1,0,134508,45725,465.62,5,0,N,2,N,N,...,0,9,105114.0,451,0,N,5817,102,0,4376
2,6881,15408,188328,513.80,5,0,N,0,N,N,...,0,6,152458.0,457,59333,N,0,102,0,483434
3,6716,157159,29967,1016.11,5,62,N,5,N,N,...,0,5,172946.0,247,50436,N,3281,102,0,1407164
4,5975,105985,81305,713.66,5,62,N,4,N,N,...,0,6,182129.0,263,93775,N,5817,102,0,1051004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3493,40382,85290,490.14,5,71,Y,8,N,N,...,0,98,170724.0,360,51357,N,413,94,0,796397
4996,6767,40382,85290,74.77,5,62,Y,8,N,N,...,0,114,161132.0,441,20181,N,5817,102,0,1258007
4997,6767,40382,85290,39.24,5,62,Y,8,N,N,...,0,100,162321.0,441,20181,N,5817,102,0,1250889
4998,6769,40382,85290,168.20,5,62,Y,8,N,N,...,0,100,162436.0,373,81206,N,5817,102,0,327739


In [16]:
Configs.CATEGORY

['ecfg',
 'flbmk',
 'flg_3dsmk',
 'insfg',
 'ovrlt',
 'scity',
 'csmcu',
 'cano',
 'mchno',
 'hcefg',
 'bacno',
 'contp',
 'etymd',
 'acqic']

In [23]:
recipe = Configs.CONAM_AGG_RECIPE_1
recipe

[(['cano'],
  [('conam', 'count'),
   ('conam', 'min'),
   ('conam', 'max'),
   ('conam', 'mean'),
   ('conam', 'median'),
   ('conam', 'var'),
   ('conam', 'sum')]),
 (['bacno', 'cano'],
  [('conam', 'count'),
   ('conam', 'min'),
   ('conam', 'max'),
   ('conam', 'mean'),
   ('conam', 'median'),
   ('conam', 'var'),
   ('conam', 'sum')])]

In [22]:
for m in range(len(recipe)):
    cols = recipe[m][0]
    print (cols)


['cano']
['bacno', 'cano']


In [34]:
recipe[0][1]

[('conam', 'count'),
 ('conam', 'min'),
 ('conam', 'max'),
 ('conam', 'mean'),
 ('conam', 'median'),
 ('conam', 'var'),
 ('conam', 'sum')]

In [46]:
recipe[0][1][0][0]
# target = conam

'conam'

In [44]:

for i in range(len(recipe[0][1])):
    print (recipe[0][1][i][1])

count
min
max
mean
median
var
sum


In [57]:
for m in range(len(recipe)):
    cols = recipe[m][0]
    print("current col: {}".format(cols), "\n")
    for n in range(len(recipe[m][1])):
        target = recipe[m][1][n][0] 
        print("current target: {}".format(target))
        
        method = recipe[m][1][n][1]
        print("current method: {}".format(method))
        
        name_grouped_target = method+"_"+target+'_BY_'+'_'.join(cols) # "_".join(list) 把list裡字串用"_"接起來
        print ("current name gruped target:{}".format(name_grouped_target))
        
        tmp = df[cols + [target]].groupby(cols).agg(method)
        print("current temp:{}".format(tmp))
        if n==0 & method == "count":
            temp = tmp
        tmp = tmp.reset_index().rename(index=str, columns={target: name_grouped_target})
        print("current tmp:{}".format(tmp))
        df_train = df_train.merge(tmp, how='left', on=cols)
        print("current df_train".format(df_train))
        df_test = df_test.merge(tmp, how='left', on=cols)
        print("current df_test".format(df_test))



current col: ['cano'] 

current target: conam
current method: count
current name gruped target:count_conam_BY_cano
current temp:        conam
cano         
44          1
84          3
93          1
109         1
168         1
...       ...
212780      1
213082      1
213113      1
213197      1
213497     37

[5175 rows x 1 columns]


TypeError: unsupported operand type(s) for &: 'int' and 'str'

In [48]:
a=111111
print("dd: {}".format(a))

dd: 111111


In [54]:
cols = ["1","2","3"]
'_'.join(cols)

'1_2_3'

In [56]:
temp

conam
bacno  cano         
70     82961       1
125    116506      2
       116507      1
128    5069        1
161    191815      1
...              ...
163706 182766      3
163712 144174      1
163797 193898      1
163798 101298      1
163858 165016      4

[5175 rows x 1 columns]

In [58]:
temp

conam
bacno  cano         
70     82961       1
125    116506      2
       116507      1
128    5069        1
161    191815      1
...              ...
163706 182766      3
163712 144174      1
163797 193898      1
163798 101298      1
163858 165016      4

[5175 rows x 1 columns]

In [60]:
m,n = 0,0
target = recipe[m][1][n][0]
print("current target: {}".format(target))

current target: conam


In [61]:
method = recipe[m][1][n][1]
print("current method: {}".format(method))

current method: count


In [62]:
name_grouped_target = method+"_"+target+'_BY_'+'_'.join(cols) # "_".join(list) 把list裡字串用"_"接起來
print ("current name gruped target:{}".format(name_grouped_target))

current name gruped target:count_conam_BY_cano


In [64]:
tmp = df[cols + [target]].groupby(cols).agg(method)
tmp

,conam
cano,
44,1
84,3
93,1
109,1
168,1
...,...
212780,1
213082,1
213113,1


In [65]:
tmp = tmp.reset_index().rename(index=str, columns={target: name_grouped_target}) # col name 從target 改到  name_grouped_target (以便merge)

In [66]:
tmp

,cano,count_conam_BY_cano
0,44,1
1,84,3
2,93,1
3,109,1
4,168,1
...,...,...
5170,212780,1
5171,213082,1
5172,213113,1
5173,213197,1


In [67]:
target


'conam'

In [68]:
name_grouped_target

'count_conam_BY_cano'

In [69]:
df_train = df_train.merge(tmp, how='left', on=cols)
df_train

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,var_conam_BY_cano_y,sum_conam_BY_cano_y,count_conam_BY_bacno_cano_y,min_conam_BY_bacno_cano_y,max_conam_BY_bacno_cano_y,mean_conam_BY_bacno_cano_y,median_conam_BY_bacno_cano_y,var_conam_BY_bacno_cano_y,sum_conam_BY_bacno_cano_y,count_conam_BY_cano
0,6881,113261,38038,513.80,5,0,N,0,N,N,...,NaN,513.80,1,513.80,513.80,513.800,513.800,NaN,513.80,1
1,0,134508,45725,465.62,5,0,N,2,N,N,...,NaN,465.62,1,465.62,465.62,465.620,465.620,NaN,465.62,1
2,6881,15408,188328,513.80,5,0,N,0,N,N,...,NaN,513.80,1,513.80,513.80,513.800,513.800,NaN,513.80,1
3,6716,157159,29967,1016.11,5,62,N,5,N,N,...,NaN,1016.11,1,1016.11,1016.11,1016.110,1016.110,NaN,1016.11,1
4,5975,105985,81305,713.66,5,62,N,4,N,N,...,NaN,713.66,1,713.66,713.66,713.660,713.660,NaN,713.66,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,6881,29914,210026,513.80,5,0,N,0,N,N,...,23263.24500,811.90,2,298.10,513.80,405.950,405.950,23263.24500,811.90,2
4996,0,21984,205835,465.62,5,0,N,2,N,N,...,6939.59805,1049.05,2,465.62,583.43,524.525,524.525,6939.59805,1049.05,2
4997,6767,68907,87723,841.46,5,62,N,5,N,N,...,NaN,841.46,1,841.46,841.46,841.460,841.460,NaN,841.46,1
4998,6769,156310,201805,346.96,5,62,N,5,N,N,...,NaN,346.96,1,346.96,346.96,346.960,346.960,NaN,346.96,1


In [70]:
df_test = df_test.merge(tmp, how='left', on=cols)

In [71]:
df_test

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,var_conam_BY_cano_y,sum_conam_BY_cano_y,count_conam_BY_bacno_cano_y,min_conam_BY_bacno_cano_y,max_conam_BY_bacno_cano_y,mean_conam_BY_bacno_cano_y,median_conam_BY_bacno_cano_y,var_conam_BY_bacno_cano_y,sum_conam_BY_bacno_cano_y,count_conam_BY_cano
0,6881,163188,116799,513.80,5,0,N,0,N,N,...,0.0000,5651.80,11,513.80,513.80,513.80,513.80,0.0000,5651.80,11
1,6881,163188,116799,513.80,5,0,N,0,N,N,...,0.0000,5651.80,11,513.80,513.80,513.80,513.80,0.0000,5651.80,11
2,6881,163188,116799,513.80,5,0,N,0,N,N,...,0.0000,5651.80,11,513.80,513.80,513.80,513.80,0.0000,5651.80,11
3,6881,163188,116799,513.80,5,0,N,0,N,N,...,0.0000,5651.80,11,513.80,513.80,513.80,513.80,0.0000,5651.80,11
4,6881,163188,116799,513.80,5,0,N,0,N,N,...,0.0000,5651.80,11,513.80,513.80,513.80,513.80,0.0000,5651.80,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3493,40382,85290,490.14,5,71,Y,8,N,N,...,31643.4289,957.85,5,39.24,490.14,191.57,168.20,31643.4289,957.85,5
4996,6767,40382,85290,74.77,5,62,Y,8,N,N,...,31643.4289,957.85,5,39.24,490.14,191.57,168.20,31643.4289,957.85,5
4997,6767,40382,85290,39.24,5,62,Y,8,N,N,...,31643.4289,957.85,5,39.24,490.14,191.57,168.20,31643.4289,957.85,5
4998,6769,40382,85290,168.20,5,62,Y,8,N,N,...,31643.4289,957.85,5,39.24,490.14,191.57,168.20,31643.4289,957.85,5


In [72]:
#針對每個recipe call 這個function
def group_target_by_cols(df_train, df_test, recipe):
    df = pd.concat([df_train, df_test], axis = 0) # concat所有data做前處理找feature
    for m in range(len(recipe)):
        cols = recipe[m][0] # groupby對象
        for n in range(len(recipe[m][1])): # 對 target apply 每個method  
            target = recipe[m][1][n][0]
            method = recipe[m][1][n][1]
            name_grouped_target = method+"_"+target+'_BY_'+'_'.join(cols)
            tmp = df[cols + [target]].groupby(cols).agg(method) 
            # grouby cols, applying method(count,min,max,mean,median,var,sum TO TARGET (conam, iterm...))
            tmp = tmp.reset_index().rename(index=str, columns={target: name_grouped_target}) # rename: target to name_grouped_target
            # prepare DF to merge with raw data : reset index, col rename 
            df_train = df_train.merge(tmp, how='left', on=cols) # how = "left", on cols ->等於自動分好train or test
            df_test = df_test.merge(tmp, how='left', on=cols)

        # reduced memory    
        del tmp
        gc.collect()
    
    return df_train, df_test